In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import csv

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,68.16,56,75,8.05,broken clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,43.48,70,100,6.89,overcast clouds
2,2,Auchel,FR,50.5034,2.4735,63.10,82,100,5.17,overcast clouds
3,3,Flin Flon,CA,54.7682,-101.8650,60.17,82,93,4.00,overcast clouds
4,4,Harpanahalli,IN,14.8000,75.9833,72.32,85,100,12.44,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you desire for your trip?"))
max_temp = float(input("What is the maximum temperature you desire for your trip?"))

What is the minimum temperature you desire for your trip?32
What is the maximum temperature you desire for your trip?100


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,68.16,56,75,8.05,broken clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,43.48,70,100,6.89,overcast clouds
2,2,Auchel,FR,50.5034,2.4735,63.10,82,100,5.17,overcast clouds
3,3,Flin Flon,CA,54.7682,-101.8650,60.17,82,93,4.00,overcast clouds
4,4,Harpanahalli,IN,14.8000,75.9833,72.32,85,100,12.44,overcast clouds
5,5,Nome,US,64.5011,-165.4064,44.67,93,100,12.66,light rain
6,6,Hermanus,ZA,-34.4187,19.2345,59.81,84,100,11.10,overcast clouds
7,7,Punta Arenas,CL,-53.1500,-70.9167,37.51,75,0,16.11,clear sky
8,8,Norsup,VU,-16.0667,167.3833,77.70,84,100,9.33,moderate rain
9,9,Hobart,AU,-42.8794,147.3294,60.26,71,75,8.05,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,68.16,56,75,8.05,broken clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,43.48,70,100,6.89,overcast clouds
2,2,Auchel,FR,50.5034,2.4735,63.10,82,100,5.17,overcast clouds
3,3,Flin Flon,CA,54.7682,-101.8650,60.17,82,93,4.00,overcast clouds
4,4,Harpanahalli,IN,14.8000,75.9833,72.32,85,100,12.44,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
703,703,Chimoio,MZ,-19.1164,33.4833,51.55,91,0,3.56,clear sky
704,704,Gryfice,PL,53.9165,15.2003,58.84,89,33,1.36,scattered clouds
705,705,Thyolo,MW,-16.0678,35.1405,52.16,97,33,1.12,scattered clouds
706,706,Trebinje,BA,42.7119,18.3442,68.36,88,0,6.91,clear sky


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Thompson,CA,68.16,broken clouds,55.7435,-97.8558,
1,Tuktoyaktuk,CA,43.48,overcast clouds,69.4541,-133.0374,
2,Auchel,FR,63.10,overcast clouds,50.5034,2.4735,
3,Flin Flon,CA,60.17,overcast clouds,54.7682,-101.8650,
4,Harpanahalli,IN,72.32,overcast clouds,14.8000,75.9833,
5,Nome,US,44.67,light rain,64.5011,-165.4064,
6,Hermanus,ZA,59.81,overcast clouds,-34.4187,19.2345,
7,Punta Arenas,CL,37.51,clear sky,-53.1500,-70.9167,
8,Norsup,VU,77.70,moderate rain,-16.0667,167.3833,
9,Hobart,AU,60.26,broken clouds,-42.8794,147.3294,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Skipping, Hotel wasn't found")

Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hotel wasn't found
Skipping, Hote

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name,Hotel name
0,Thompson,CA,68.16,broken clouds,55.7435,-97.8558,,Burntwood Hotel
1,Tuktoyaktuk,CA,43.48,overcast clouds,69.4541,-133.0374,,End of The Road Ltd.
2,Auchel,FR,63.10,overcast clouds,50.5034,2.4735,,Gîte d’étape du Presbytère
3,Flin Flon,CA,60.17,overcast clouds,54.7682,-101.8650,,Victoria Inn Flin Flon
4,Harpanahalli,IN,72.32,overcast clouds,14.8000,75.9833,,Bismillah Biriyani Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))